In [ ]:
pip install hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 21.0 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=5ea460254a83c778ab103dc7b5aa42edb0c087b127bc233022130e77ca6f263d
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
   

In [ ]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
import string
import re
import hazm



class PreProcess:
    def __init__(self) -> None:
        persian_punctuations = '''`÷×؛#<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
        self.punctuations_list = string.punctuation + persian_punctuations
        self.arabic_diacritics = re.compile("""
                                ّ    | # Tashdid
                                َ    | # Fatha
                                ً    | # Tanwin Fath
                                ُ    | # Damma
                                ٌ    | # Tanwin Damm
                                ِ    | # Kasra
                                ٍ    | # Tanwin Kasr
                                ْ    | # Sukun
                                ـ     # Tatwil/Kashida
                            """, re.VERBOSE)
        self.stop_words = hazm.stopwords_list()
        self.lemmatizer = hazm.Lemmatizer()

    def remove_punctuations(self, text : str) -> str:
        translator = str.maketrans('', '', self.punctuations_list)
        return text.translate(translator)

    def remove_diacritics(self, text : str) -> str:
        text = re.sub(self.arabic_diacritics, '', text)
        return text

    def normalize_persian(self, text : str) -> str:
        text = re.sub("[إأآا]", "ا", text)
        text = re.sub("ي", "ی", text)
        text = re.sub("ؤ", "و", text)
        text = re.sub("ئ", "ی", text)
        text = re.sub("ة", "ه", text)
        text = re.sub("ك" ,"ک" , text)
        text = re.sub("[^ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]", " ", text)
        text = re.sub("[^\S\n\t]+", ' ', text)
        return text

    def remove_repeating_char(self, text : str) -> str:
        return re.sub(r'(.)\1+', r'\1', text)

    def tokenize(self, text : str) -> list:
        return hazm.word_tokenize(text)

    def remove_stopwords(self, tokens : list) -> list:
        return [token for token in tokens if token not in self.stop_words]

    def lemmatizer_text(self, tokens : list) -> list:
        return [self.lemmatizer.lemmatize(token) for token in tokens]


    def pipeline(self, text : str) -> list:
        text = self.remove_punctuations(text)
        text = self.remove_diacritics(text)
        text = self.normalize_persian(text)
        text = self.remove_repeating_char(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        tokens = self.lemmatizer_text(tokens)
        return tokens

In [ ]:
# بارگذاری داده
dataset = load_dataset("PNLPhub/snappfood-sentiment-analysis")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/447 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/8.93M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52110 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8337 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9033 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'label', 'label_id'],
        num_rows: 52110
    })
    validation: Dataset({
        features: ['comment', 'label', 'label_id'],
        num_rows: 8337
    })
    test: Dataset({
        features: ['comment', 'label', 'label_id'],
        num_rows: 9033
    })
})

In [ ]:
dataset = dataset['train']
comments = dataset['comment']
labels = dataset['label']

In [ ]:
comments

['غذا خیلی سرد بود در صورتیکه فاصله ما خیلی کم است',
 'بهتره بتونیم ران یا سینه رو خودمون انتخاب کنیم',
 'غذا بد بود حالم خیییییلی بده. دل دردو دل پیچه. معلوم نیست چی توش ریختن. افتضاااااااااح بود. دوساعته ک حالم بده.',
 'با سلام سابق بر این بسته بندی از کیفیت بهتری برخوردار بود. ولی در حال حاضر ایراداتی وجود داره. به طور نمونه نوشابه قوطی فاقد نی هست، چنگال برای سرو سیب زمینی نداره، دستمال کاغذی هم در بسته بندی نیست.',
 'سلام، خیلی ممنون و متشکرم',
 'خیلی زود رسید و ممنون م ازشون',
 'مواد پیتزا بسیااار کم بود با اینکه قیمت اصلا پایین نیست. ولی کلا طعم پیتزا خوب و تازه بود',
 'کیفیت زیاد خوب نبود،',
 'مرغی که واسه ما آوردن بو میداد انگار که مونده بود!',
 'همه چی خوب بود. ولی اگه تخفیف نداشتم اصلا خرید نمی\u200cکردم چون گرونه برام:)',
 'اصلا خوب نبود و اصلا مخلفات نداشت خمیر خمیر بود من راضی نبودم',
 'غذا بسیار لذیذ با طعمی فوق العاده بود جوجه بسیار خوشمزه و کباب بینظیر بود. فقط برنج کمی شفته شده بود که مزه غذا این کمیت رو از بین برد و مزه را بینظیر نشون داد سوپ بحدی عالی بود که خیلی دو

In [ ]:
labels

['SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'SAD',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'SAD',
 'SAD',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SAD',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'SAD',
 'HAPPY',
 'HAPPY',
 'HAPPY',
 'SA

In [ ]:
# اگر لیبل‌ها رشته‌ای هستند:
if isinstance(labels[0], str):
    label_map = {"HAPPY": 1, "SAD": 0}
    labels = [label_map[l] for l in labels]

In [ ]:
# comments pre-processing
pre_process = PreProcess()
comments_prep = []
for item in comments:
  c_p_p = PreProcess.pipeline(pre_process, item)
  comments_prep.append(c_p_p)

In [ ]:
comments_prep

[['غذا', 'سرد', 'صورتیکه', 'فاصله'],
 ['بهتره', 'بتون', 'ران', 'سینه', 'خودمون', 'انتخاب'],
 ['غذا',
  'بد',
  'حالم',
  'بده',
  'دل',
  'دردو',
  'دل',
  'پیچه',
  'معلوم',
  'چی',
  'توش',
  'ریختن',
  'افتضاح',
  'دوساعته',
  'ک',
  'حالم',
  'بده'],
 ['سلام',
  'سابق',
  'بسته',
  'کیفیت',
  'بهتری',
  'برخوردار',
  'حاضر',
  'ایرادات',
  'داره',
  'نمونه',
  'نوشابه',
  'قوطی',
  'فاقد',
  'نی',
  'هست',
  'چنگال',
  'سرو',
  'سیب',
  'زمینی',
  'نداره',
  'دستمال',
  'کاغذی',
  'بسته'],
 ['سلام', 'منون', 'متشکر'],
 ['زود', 'منون', '#هست', 'ازشون'],
 ['مواد', 'پیتزا', 'قیمت', 'اصلا', 'پاین', 'کلا', 'طعم', 'پیتزا', 'تازه'],
 ['کیفیت'],
 ['مرغی', 'واسه', 'اوردن', 'بو', 'میداد', 'انگار', 'موند#مون'],
 ['چی', 'اگه', 'تخفیف', 'داشت#دار', 'اصلا', 'خرید', 'نمی', 'گرونه', 'برام'],
 ['اصلا', 'اصلا', 'مخلفات', 'داشت#دار', 'خمیر', 'خمیر', 'راضی', 'بود#است'],
 ['غذا',
  'لذیذ',
  'طعم',
  'العاده',
  'جوجه',
  'خوشمزه',
  'کباب',
  'بینظیر',
  'برنج',
  'شفته',
  'مزه',
  'غذا',
  'کمیت',
  

In [ ]:
comments = [" ".join(tokens) for tokens in comments_prep]

In [ ]:
comments

['غذا سرد صورتیکه فاصله',
 'بهتره بتون ران سینه خودمون انتخاب',
 'غذا بد حالم بده دل دردو دل پیچه معلوم چی توش ریختن افتضاح دوساعته ک حالم بده',
 'سلام سابق بسته کیفیت بهتری برخوردار حاضر ایرادات داره نمونه نوشابه قوطی فاقد نی هست چنگال سرو سیب زمینی نداره دستمال کاغذی بسته',
 'سلام منون متشکر',
 'زود منون #هست ازشون',
 'مواد پیتزا قیمت اصلا پاین کلا طعم پیتزا تازه',
 'کیفیت',
 'مرغی واسه اوردن بو میداد انگار موند#مون',
 'چی اگه تخفیف داشت#دار اصلا خرید نمی گرونه برام',
 'اصلا اصلا مخلفات داشت#دار خمیر خمیر راضی بود#است',
 'غذا لذیذ طعم العاده جوجه خوشمزه کباب بینظیر برنج شفته مزه غذا کمیت برد مزه بینظیر نشون سوپ بحدی دوست داشت#دار باخت#باز خرید#خر خورد#خور خواهشا منوتون سوپ کیلو زارید#زار مدیر محترم کارکنان عزیز غنچه سپاسگزار',
 'تحویل اسنپ فود دیر غذا سرد اصلا دلیوری راضی بود#است',
 'نمی رندید#رند رفتاری بتونه غذای افتضاح اماده کنه سوپ مزخرف کته مزخرف خورشت مزخرف سالاد شیرازی ابلیموی واقعا اسم برد#بر متاسف',
 'کیفیت پاین تر پاین یه ساندویچ تومنی',
 'داغ تازه خوش عطر دستم ازتون خواست#

In [ ]:
dataset = {"comments": comments, "label": labels}
dataset

{'comments': ['غذا سرد صورتیکه فاصله',
  'بهتره بتون ران سینه خودمون انتخاب',
  'غذا بد حالم بده دل دردو دل پیچه معلوم چی توش ریختن افتضاح دوساعته ک حالم بده',
  'سلام سابق بسته کیفیت بهتری برخوردار حاضر ایرادات داره نمونه نوشابه قوطی فاقد نی هست چنگال سرو سیب زمینی نداره دستمال کاغذی بسته',
  'سلام منون متشکر',
  'زود منون #هست ازشون',
  'مواد پیتزا قیمت اصلا پاین کلا طعم پیتزا تازه',
  'کیفیت',
  'مرغی واسه اوردن بو میداد انگار موند#مون',
  'چی اگه تخفیف داشت#دار اصلا خرید نمی گرونه برام',
  'اصلا اصلا مخلفات داشت#دار خمیر خمیر راضی بود#است',
  'غذا لذیذ طعم العاده جوجه خوشمزه کباب بینظیر برنج شفته مزه غذا کمیت برد مزه بینظیر نشون سوپ بحدی دوست داشت#دار باخت#باز خرید#خر خورد#خور خواهشا منوتون سوپ کیلو زارید#زار مدیر محترم کارکنان عزیز غنچه سپاسگزار',
  'تحویل اسنپ فود دیر غذا سرد اصلا دلیوری راضی بود#است',
  'نمی رندید#رند رفتاری بتونه غذای افتضاح اماده کنه سوپ مزخرف کته مزخرف خورشت مزخرف سالاد شیرازی ابلیموی واقعا اسم برد#بر متاسف',
  'کیفیت پاین تر پاین یه ساندویچ تومنی',
  'داغ تا

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import os

file_path = '/content/drive/My Drive/snappfood_pre_processed_dataset.csv'

try:
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write the header row
        writer.writerow(dataset.keys())
        # Write the data rows
        writer.writerows(zip(*dataset.values()))
    print(f"Dictionary successfully written to {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Dictionary successfully written to /content/drive/My Drive/snappfood_pre_processed_dataset.csv
